In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('dadosOrganizados.xlsx')

In [3]:
df['paciente_idade'] = pd.to_numeric(df['paciente_idade'].str.extract(r'(\d+)')[0], errors='coerce')
df['paciente_estado'] = df['paciente_cidade'].str.split('/').str[1]

In [4]:
df.columns

Index(['Unnamed: 0', 'estado', 'id', 'data', 'tecnologia', 'CID',
       'natjus_responsavel', 'status', 'link_site', 'link_PDF', 'nota_id',
       'conclusao', 'paciente_idade', 'paciente_genero', 'paciente_cidade',
       'advogado_nome', 'advogado_numOAB', 'advogado_instituicao',
       'processo_justica', 'processo_vara', 'diagnostico_cid',
       'diagnostico_nome', 'tecnologia_tipo', 'tecnologia_registro',
       'tecnologia_comercial', 'tecnologia_principio',
       'tecnologia_viaAdministracao', 'tecnologia_posologia',
       'tecnologia_continuo', 'tecnologia_offlabel', 'tecnologia_PCDT',
       'tecnologia_incorporacao', 'tecnologia_ondeSUS',
       'tecnologia_oncologico', 'outrasTec_generico', 'outrasTec_similar',
       'evidencias_CONITEC', 'conclusao_resultado', 'conclusao_motivacao',
       'conclusao_evidencias', 'conclusao_urgencia', 'conclusao_anexos',
       'inst_responsavel', 'apoio_tutoria', 'paciente_estado'],
      dtype='object')

In [5]:
df.processo_justica.value_counts()

processo_justica
Justiça Estadual    16915
Justiça Federal      3343
Name: count, dtype: int64

In [6]:
# Agrupar por estado
grupo_estado = df.groupby('estado')

# Número de pacientes únicos por estado
n_pacientes_unicos = grupo_estado['link_site'].nunique()

# Número de pareceres na base
n_pareceres = grupo_estado.size()

# Idade média
idade_media = grupo_estado['paciente_idade'].mean().round(1)

# Percentual de mulheres
percentual_mulheres = grupo_estado.apply(lambda x: (x['paciente_genero'] == 'Feminino').mean() * 100).round(1)

# Representação por instituição responsável
representacao_defensoria = grupo_estado.apply(lambda x: (x['advogado_instituicao'] == 'Defensoria Pública').mean() * 100).round(1)
representacao_ministerio_publico = grupo_estado.apply(lambda x: (x['advogado_instituicao'] == 'Ministério Público').mean() * 100).round(1)
representacao_justica_federal = grupo_estado.apply(lambda x: (x['processo_justica'] == 'Justiça Federal').mean() * 100).round(1)

# Construir o DataFrame final
tabela_final = pd.DataFrame({
    'Nº de pacientes únicos': n_pacientes_unicos,
    'Nº de pareceres na base': n_pareceres,
    'Pedidos por paciente': (n_pareceres/n_pacientes_unicos).round(2),
    'Idade média': idade_media,
    '% de mulheres': percentual_mulheres,
    '% de representação pela Defensoria Pública': representacao_defensoria,
    '% de representação pelo Ministério Público': representacao_ministerio_publico,
    '% de pareceres da Justiça Federal': representacao_justica_federal
})

# Exibir a tabela final
tabela_final.reset_index(inplace=True)#
tabela_final.to_excel('tabela_descritiva.xlsx')

C:\Users\bruno\AppData\Local\Temp\ipykernel_16808\3292427347.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentual_mulheres = grupo_estado.apply(lambda x: (x['paciente_genero'] == 'Feminino').mean() * 100).round(1)
C:\Users\bruno\AppData\Local\Temp\ipykernel_16808\3292427347.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  representacao_defensoria = grupo_estado.apply(lambda x: (x['advogado_in

In [7]:
df.tecnologia_continuo.value_counts()

tecnologia_continuo
Sim              9321
Não informado    2798
Não              1627
Name: count, dtype: int64

In [8]:
colunas = ['tecnologia_registro', 'tecnologia_continuo', 'tecnologia_offlabel',
    'tecnologia_PCDT', 'tecnologia_incorporacao',
    'tecnologia_oncologico', 'outrasTec_generico', 'outrasTec_similar'
]

colunas2 = ['tecnologia_continuo', 'tecnologia_offlabel',
    'tecnologia_PCDT', 'tecnologia_incorporacao',
    'tecnologia_oncologico']

colunas3 = ['tecnologia_registro', 'tecnologia_continuo', 'tecnologia_offlabel',
    'tecnologia_PCDT', 'tecnologia_incorporacao',
    'tecnologia_oncologico', 'outrasTec_generico', 'outrasTec_similar', 'evidencias_CONITEC'
]

In [9]:
for coluna in colunas:
    print(df[coluna].value_counts())

tecnologia_registro
Sim    19730
Não      569
Name: count, dtype: int64
tecnologia_continuo
Sim              9321
Não informado    2798
Não              1627
Name: count, dtype: int64
tecnologia_offlabel
Sim              16944
Não               2128
Não informado     1054
Não sabe           173
Name: count, dtype: int64
tecnologia_PCDT
Não              14365
Sim               4491
Não informado     1374
Não sabe            68
Name: count, dtype: int64
tecnologia_incorporacao
Não              13875
Sim               5796
Não informado      412
Não sabe           214
Name: count, dtype: int64
tecnologia_oncologico
Não              16140
Sim               4105
Não informado       43
Não sabe            11
Name: count, dtype: int64
outrasTec_generico
Não    14589
Sim     5668
Name: count, dtype: int64
outrasTec_similar
Não    12967
Sim     7290
Name: count, dtype: int64


In [10]:
# Agrupamento por 'estado'
grupo_estado = df.groupby('estado')

# Função para calcular o percentual de 'Sim' para uma coluna
def calcular_percentual(coluna):
    return grupo_estado.apply(lambda x: (x[coluna] == 'Sim').mean() * 100).round(1)

# Dicionário para armazenar os percentuais de cada coluna
percentuais = {}

# Loop para calcular o percentual de 'Sim' para cada coluna e adicionar ao dicionário
for coluna in colunas:
    percentuais[f'% de {coluna}'] = calcular_percentual(coluna)

percentuais['% de não recomendado pela Conitec'] = grupo_estado.apply(lambda x: (x['evidencias_CONITEC'] == 'Não Recomendada').mean() * 100).round(1)
percentuais['% de não avaliado pela Conitec'] = grupo_estado.apply(lambda x: (x['evidencias_CONITEC'] == 'Não avaliada').mean() * 100).round(1)
percentuais['% de recomendado pela Conitec'] = grupo_estado.apply(lambda x: (x['evidencias_CONITEC'] == 'Recomendada').mean() * 100).round(1)

# Construir o DataFrame final
tabela_final = pd.DataFrame(percentuais)

# Exibir o DataFrame final
tabela_final


C:\Users\bruno\AppData\Local\Temp\ipykernel_16808\1636412511.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return grupo_estado.apply(lambda x: (x[coluna] == 'Sim').mean() * 100).round(1)
C:\Users\bruno\AppData\Local\Temp\ipykernel_16808\1636412511.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  percentuais['% de não recomendado pela Conitec'] = grupo_estado.apply(lambda x: (x['evidencias_CONITEC'

,% de tecnologia_registro,% de tecnologia_continuo,% de tecnologia_offlabel,% de tecnologia_PCDT,% de tecnologia_incorporacao,% de tecnologia_oncologico,% de outrasTec_generico,% de outrasTec_similar,% de não recomendado pela Conitec,% de não avaliado pela Conitec,% de recomendado pela Conitec
estado,,,,,,,,,,,
BA - Bahia,89.8,0.1,92.0,29.5,25.3,24.6,0.1,0.3,4.9,88.8,6.3
DF - Distrito Federal,97.9,7.1,81.9,12.8,18.4,30.6,27.5,34.5,31.3,51.5,17.2
ES - Espírito Santo,98.4,52.3,79.0,25.7,39.7,10.2,47.2,58.0,34.4,46.2,19.4
GO - Goiás,99.8,84.0,78.5,21.3,50.8,18.4,17.1,37.8,24.1,51.2,24.7
MA - Maranhão,98.2,66.0,90.3,45.9,54.8,6.6,57.3,63.2,22.5,35.4,42.1
MS - Mato Grosso do Sul,99.5,82.0,89.0,20.1,29.8,12.4,40.0,53.0,22.2,63.7,14.1
MT - Mato Grosso,99.9,50.2,94.1,13.8,28.9,20.6,31.7,50.9,40.7,42.8,16.5
Nacional,99.7,49.2,92.7,18.8,26.9,20.7,42.7,53.4,24.3,56.6,19.1
PE - Pernambuco,95.3,54.4,92.0,29.6,37.4,41.1,36.9,41.9,17.9,61.8,20.3


In [11]:
# Agrupamento por 'estado'
grupo_estado = df.groupby('estado')

# Função para calcular o percentual de 'Não informado' ou 'Não sabe' para uma coluna
def calcular_percentual(coluna):
    return grupo_estado.apply(lambda x: ((x[coluna] == 'Não informado') | (x[coluna] == 'Não sabe')).mean() * 100).round(1)

# Dicionário para armazenar os percentuais de cada coluna
percentuais = {}

# Loop para calcular o percentual de 'Não informado' ou 'Não sabe' para cada coluna e adicionar ao dicionário
for coluna in colunas2:
    percentuais[f'% de {coluna} - Não informado/Não sabe'] = calcular_percentual(coluna)

# Construir o DataFrame final
tabela_final = pd.DataFrame(percentuais)

# Exibir o DataFrame final
tabela_final


C:\Users\bruno\AppData\Local\Temp\ipykernel_16808\117459205.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return grupo_estado.apply(lambda x: ((x[coluna] == 'Não informado') | (x[coluna] == 'Não sabe')).mean() * 100).round(1)


,% de tecnologia_continuo - Não informado/Não sabe,% de tecnologia_offlabel - Não informado/Não sabe,% de tecnologia_PCDT - Não informado/Não sabe,% de tecnologia_incorporacao - Não informado/Não sabe,% de tecnologia_oncologico - Não informado/Não sabe
estado,,,,,
BA - Bahia,0.1,0.0,15.7,7.9,1.0
DF - Distrito Federal,0.1,0.4,0.4,0.1,0.3
ES - Espírito Santo,20.0,10.7,3.5,0.9,0.1
GO - Goiás,9.0,13.3,5.4,2.2,0.2
MA - Maranhão,18.9,1.5,5.6,0.5,0.4
MS - Mato Grosso do Sul,13.8,7.3,7.2,1.6,0.3
MT - Mato Grosso,0.3,0.3,0.1,0.0,0.0
Nacional,18.6,1.2,0.5,0.1,0.1
PE - Pernambuco,7.9,1.3,0.6,1.9,0.0


In [12]:
df.columns

Index(['Unnamed: 0', 'estado', 'id', 'data', 'tecnologia', 'CID',
       'natjus_responsavel', 'status', 'link_site', 'link_PDF', 'nota_id',
       'conclusao', 'paciente_idade', 'paciente_genero', 'paciente_cidade',
       'advogado_nome', 'advogado_numOAB', 'advogado_instituicao',
       'processo_justica', 'processo_vara', 'diagnostico_cid',
       'diagnostico_nome', 'tecnologia_tipo', 'tecnologia_registro',
       'tecnologia_comercial', 'tecnologia_principio',
       'tecnologia_viaAdministracao', 'tecnologia_posologia',
       'tecnologia_continuo', 'tecnologia_offlabel', 'tecnologia_PCDT',
       'tecnologia_incorporacao', 'tecnologia_ondeSUS',
       'tecnologia_oncologico', 'outrasTec_generico', 'outrasTec_similar',
       'evidencias_CONITEC', 'conclusao_resultado', 'conclusao_motivacao',
       'conclusao_evidencias', 'conclusao_urgencia', 'conclusao_anexos',
       'inst_responsavel', 'apoio_tutoria', 'paciente_estado'],
      dtype='object')

In [13]:
'natjus_responsavel', 'paciente_idade', 'paciente_genero', 'advogado_instituicao',
'diagnostico_cid', 'tecnologia_registro', 'tecnologia_principio',
'tecnologia_continuo', 'tecnologia_offlabel', 'tecnologia_PCDT',
'tecnologia_incorporacao', 'tecnologia_ondeSUS',
'tecnologia_oncologico', 'outrasTec_generico', 'outrasTec_similar',
'evidencias_CONITEC'

'evidencias_CONITEC'

In [14]:
for coluna in colunas3:
    # Criando a tabela dinâmica com contagem
    pivot_df = df.pivot_table(
        index=['estado', coluna, 'conclusao_resultado'],
        aggfunc='size'
    ).reset_index(name='count')

    # Contando o total de registros por 'estado' e 'tecnologia_registro'
    total_counts = pivot_df.groupby(['estado', coluna])['count'].transform('sum')

    # Substituindo contagens por NA onde o total é menor que 10
    pivot_df['count'] = np.where(total_counts < 10, np.nan, pivot_df['count'])

    # Calculando as porcentagens e formatando a string
    pivot_df['formatted'] = np.where(
        pivot_df['count'].isna(),
        np.nan,
        pivot_df['count'] / total_counts * 100
    )
    pivot_df['formatted'] = pivot_df['formatted'].round(1).astype(str) + '% (n = ' + pivot_df['count'].astype(str) + ')'

    # Exibindo o resultado com a coluna formatada
    pivot_df = pivot_df.drop(columns='count')
    pivot_df.to_excel(f'{coluna}.xlsx')

In [15]:
df.columns

Index(['Unnamed: 0', 'estado', 'id', 'data', 'tecnologia', 'CID',
       'natjus_responsavel', 'status', 'link_site', 'link_PDF', 'nota_id',
       'conclusao', 'paciente_idade', 'paciente_genero', 'paciente_cidade',
       'advogado_nome', 'advogado_numOAB', 'advogado_instituicao',
       'processo_justica', 'processo_vara', 'diagnostico_cid',
       'diagnostico_nome', 'tecnologia_tipo', 'tecnologia_registro',
       'tecnologia_comercial', 'tecnologia_principio',
       'tecnologia_viaAdministracao', 'tecnologia_posologia',
       'tecnologia_continuo', 'tecnologia_offlabel', 'tecnologia_PCDT',
       'tecnologia_incorporacao', 'tecnologia_ondeSUS',
       'tecnologia_oncologico', 'outrasTec_generico', 'outrasTec_similar',
       'evidencias_CONITEC', 'conclusao_resultado', 'conclusao_motivacao',
       'conclusao_evidencias', 'conclusao_urgencia', 'conclusao_anexos',
       'inst_responsavel', 'apoio_tutoria', 'paciente_estado'],
      dtype='object')

In [16]:
df['CID2'] = df['CID'].str.split(' - ').str[0]
df['CID3'] = df['CID2'].str.split('.').str[0]
df['CID4-Letra'] = df['CID3'].str.extract('(\D)', expand=False)
df['CID4-Num'] = df['CID3'].str.extract('(\d+)', expand=False)

In [17]:
import pandas as pd

mapa_CID = {
    ('A', 'B'): 'Algumas doenças infecciosas e parasitárias',
    ('C', 'C'): 'Neoplasias (tumores)',
    ('E', 'E'): 'Doenças endócrinas, nutricionais e metabólicas',
    ('F', 'F'): 'Transtornos mentais e comportamentais',
    ('G', 'G'): 'Doenças do sistema nervoso',
    ('H', 'H'): 'Doenças do olho e anexos',  # adicionaremos uma condição extra para este caso
    ('I', 'I'): 'Doenças do sistema circulatório',
    ('J', 'J'): 'Doenças do sistema respiratório',
    ('K', 'K'): 'Doenças do sistema digestivo',
    ('L', 'L'): 'Doenças da pele e do tecido subcutâneo',
    ('M', 'M'): 'Doenças do sistema osteomuscular e do tecido conjuntivo',
    ('N', 'N'): 'Doenças do sistema geniturinário',
    ('O', 'O'): 'Gravidez, parto e o puerpério',
    ('P', 'P'): 'Algumas afecções originadas no período perinatal',
    ('Q', 'Q'): 'Malformações congênitas, deformidades e anomalias cromossômicas',
    ('R', 'R'): 'Sintomas, sinais e achados anormais de exames clínicos e de laboratório, não classificados em outra parte',
    ('S', 'T'): 'Lesões, envenenamentos e algumas outras consequências de causas externas',
    ('U', 'U'): 'Códigos para propósitos especiais',
    ('V', 'Y'): 'Códigos de causas externas de morbidade e de mortalidade',
    ('Z', 'Z'): 'Fatores que influenciam o estado de saúde e o contato com os serviços de saúde',
}

def obter_tipo_do_cid(linha):
    letra = linha['CID4-Letra']
    num = linha['CID4-Num']
    
    if pd.isna(num):
        return 'Outros'
    
    num = int(num)
    
    for key, value in mapa_CID.items():
        if letra >= key[0] and letra <= key[1]:
            if letra == 'H':
                if num <= 59:
                    return 'Doenças do olho e anexos'
                else:
                    return 'Doenças do ouvido e da apófise mastoide'
            elif letra == 'D':
                if num <= 48:
                    return 'Neoplasias (tumores)'
                else:
                    return 'Doenças do sangue e dos órgãos hematopoéticos e alguns transtornos envolvendo o sistema imunológico'
            else:
                return value
    return 'Outros'

df['CID4-Tipo'] = df.apply(obter_tipo_do_cid, axis=1)


In [18]:
df['CID4-Tipo'].value_counts()

CID4-Tipo
Neoplasias (tumores)                                                                                         4014
Transtornos mentais e comportamentais                                                                        3353
Doenças endócrinas, nutricionais e metabólicas                                                               2804
Doenças do sistema circulatório                                                                              1976
Doenças do sistema nervoso                                                                                   1660
Doenças do sistema osteomuscular e do tecido conjuntivo                                                      1519
Doenças do sistema respiratório                                                                              1093
Doenças da pele e do tecido subcutâneo                                                                        784
Doenças do olho e anexos                                                      

In [19]:
print(df['CID3'].value_counts(dropna=False, normalize=False).head(10))

CID3
F84    787
E11    761
C50    710
E10    708
F90    586
I50    568
J84    552
G40    518
C61    423
L20    368
Name: count, dtype: int64


In [20]:
# Agrupar por 'estado' e encontrar os 3 valores mais frequentes de 'CID4-Tipo'
top_3 = df.groupby('estado')['CID4-Tipo'] \
    .apply(lambda x: x.value_counts().nlargest(3).index.tolist()) \
    .reset_index()

# Renomear a coluna para indicar que contém os top 3 mais frequentes
top_3.columns = ['estado', 'top_3_CID4-Tipo']

# Expandir o resultado para ter uma coluna para cada um dos três valores mais frequentes
top_3 = top_3.join(pd.DataFrame(top_3.pop('top_3_CID4-Tipo').tolist(), index=top_3.index, columns=['top1', 'top2', 'top3']))

top_3.to_excel('grupos_doencas.xlsx')

In [21]:
# Agrupar por 'estado' e encontrar os 3 valores mais frequentes de 'CID4-Tipo'
top_3 = df.groupby('estado')['CID3'] \
    .apply(lambda x: x.value_counts().nlargest(10).index.tolist()) \
    .reset_index()

# Renomear a coluna para indicar que contém os top 3 mais frequentes
top_3.columns = ['estado', 'top_10_CID3']

# Expandir o resultado para ter uma coluna para cada um dos três valores mais frequentes
top_3 = top_3.join(pd.DataFrame(top_3.pop('top_10_CID3').tolist(), index=top_3.index, columns=['top1', 'top2', 'top3', 'top4', 'top5', 'top6', 'top7', 'top8', 'top9', 'top10']))

top_3.to_excel('doencas_mais_frequentes.xlsx')

In [22]:
df.loc[df['estado'] == 'Nacional']['paciente_estado'].value_counts().to_excel('natjus_nacional.xlsx')

In [23]:
for natjus in df['natjus_responsavel'].unique():
    print(natjus)
    print(df.loc[df['natjus_responsavel'] == natjus]['conclusao_resultado'].value_counts(normalize=True))

DF - Distrito Federal
conclusao_resultado
Favorável        0.614562
Não favorável    0.385438
Name: proportion, dtype: float64
MA - Maranhão
conclusao_resultado
Favorável        0.548769
Não favorável    0.451231
Name: proportion, dtype: float64
Nacional
conclusao_resultado
Não favorável    0.712111
Favorável        0.287889
Name: proportion, dtype: float64
PE - Pernambuco
conclusao_resultado
Não favorável    0.50906
Favorável        0.49094
Name: proportion, dtype: float64
RN - Rio Grande do Norte
conclusao_resultado
Não favorável    0.601494
Favorável        0.398506
Name: proportion, dtype: float64
ES - Espírito Santo
conclusao_resultado
Não favorável    0.89215
Favorável        0.10785
Name: proportion, dtype: float64
GO - Goiás
conclusao_resultado
Favorável        0.665198
Não favorável    0.334802
Name: proportion, dtype: float64
PR - Paraná
conclusao_resultado
Não favorável    0.6004
Favorável        0.3996
Name: proportion, dtype: float64
SE - Sergipe
conclusao_resultado
Favorá

In [24]:
import pandas as pd

# Assumindo que 'df' é o seu DataFrame original
# Se não for, você precisa carregar seus dados primeiro:
# df = pd.read_csv('seu_arquivo.csv')  # ou pd.read_excel('seu_arquivo.xlsx')

# Agrupar os dados e contar
grouped = df.groupby(['natjus_responsavel', 'conclusao_resultado']).size().unstack(fill_value=0)

# Calcular as proporções e converter para porcentagem
proportions = grouped.div(grouped.sum(axis=1), axis=0) * 100

# Arredondar para duas casas decimais
proportions = proportions.round(2)

# Adicionar uma coluna com o total de casos por natjus_responsavel
proportions['Total_Casos'] = grouped.sum(axis=1)

# Ordenar o DataFrame pelo total de casos (opcional)
proportions = proportions.sort_values('Total_Casos', ascending=False)

# Resetar o índice para tornar 'natjus_responsavel' uma coluna
proportions = proportions.reset_index()

# Função para formatar as colunas de porcentagem
def format_percentage(x):
    return f"{x:.2f}%" if pd.notnull(x) else ""

# Aplicar a formatação de porcentagem
for col in proportions.columns:
    if col not in ['natjus_responsavel', 'Total_Casos']:
        proportions[col] = proportions[col].apply(format_percentage)

# Salvar em Excel
with pd.ExcelWriter('output_organizado_percentual.xlsx') as writer:
    proportions.to_excel(writer, sheet_name='Porcentagens', index=False)
    grouped.reset_index().to_excel(writer, sheet_name='Contagens', index=False)

print("Planilha 'output_organizado_percentual.xlsx' gerada com sucesso!")

# Exibir as primeiras linhas do resultado para verificação
print("\nPrimeiras linhas das porcentagens:")
print(proportions.head())

Planilha 'output_organizado_percentual.xlsx' gerada com sucesso!

Primeiras linhas das porcentagens:
conclusao_resultado     natjus_responsavel Favorável Não favorável  \
0                         MT - Mato Grosso     6.33%        93.67%   
1                           SP - São Paulo    25.68%        74.32%   
2                              PR - Paraná    39.96%        60.04%   
3                    DF - Distrito Federal    61.46%        38.54%   
4                      RJ - Rio de Janeiro    70.94%        29.06%   

conclusao_resultado  Total_Casos  
0                           1500  
1                           1499  
2                           1499  
3                           1497  
4                           1497  


In [25]:
df.tecnologia_principio.value_counts().head(10)

tecnologia_principio
ESILATO DE NINTEDANIBE             419
ARIPIPRAZOL                        388
TETRAIDROCANABINOL + CANABIDIOL    387
PEMBROLIZUMABE                     383
DUPILUMABE                         369
SOMATROPINA                        364
DIMESILATO DE LISDEXANFETAMINA     352
RITUXIMABE                         311
RIVAROXABANA                       308
DAPAGLIFLOZINA                     293
Name: count, dtype: int64

In [26]:
for coluna in colunas3:
    # Criando a tabela dinâmica com contagem
    pivot_df = df.pivot_table(
        index=['estado', coluna, 'conclusao_resultado'],
        aggfunc='size'
    ).reset_index(name='count')

    # Contando o total de registros por 'estado' e 'tecnologia_registro'
    total_counts = pivot_df.groupby(['estado', coluna])['count'].transform('sum')

    # Substituindo contagens por NA onde o total é menor que 10
    pivot_df['count'] = np.where(total_counts < 10, np.nan, pivot_df['count'])

    # Calculando as porcentagens e formatando a string
    pivot_df['formatted'] = np.where(
        pivot_df['count'].isna(),
        np.nan,
        pivot_df['count'] / total_counts * 100
    )
    pivot_df['formatted'] = pivot_df['formatted'].round(1).astype(str) + '% (n = ' + pivot_df['count'].astype(str) + ')'

    # Exibindo o resultado com a coluna formatada
    pivot_df = pivot_df.drop(columns='count')
    pivot_df.to_excel(f'{coluna}.xlsx')

In [28]:
import pandas as pd
import numpy as np

# Assumindo que 'df' é seu DataFrame original
# Primeiro, vamos obter os 10 itens mais frequentes de 'tecnologia_principio'
top_10_tecnologias = df['tecnologia_principio'].value_counts().nlargest(5).index

# Filtrar o DataFrame para incluir apenas essas 10 tecnologias
df_filtered = df[df['tecnologia_principio'].isin(top_10_tecnologias)]

# Criando a tabela dinâmica com contagem
pivot_df = df_filtered.pivot_table(
    index=['estado', 'tecnologia_principio', 'conclusao_resultado'],
    aggfunc='size'
).reset_index(name='count')

# Contando o total de registros por 'estado' e 'tecnologia_principio'
total_counts = pivot_df.groupby(['estado', 'tecnologia_principio'])['count'].transform('sum')

# Substituindo contagens por NA onde o total é menor que 10
pivot_df['count'] = np.where(total_counts < 10, np.nan, pivot_df['count'])

# Calculando as porcentagens e formatando a string
pivot_df['formatted'] = np.where(
    pivot_df['count'].isna(),
    np.nan,
    pivot_df['count'] / total_counts * 100
)
pivot_df['formatted'] = pivot_df['formatted'].round(1).astype(str) + '% (n = ' + pivot_df['count'].fillna('').astype(str) + ')'

# Exibindo o resultado com a coluna formatada
pivot_df = pivot_df.drop(columns='count')
pivot_df.to_excel('top_5_tecnologia_principio.xlsx')

In [30]:
df['tecnologia_principio'].value_counts()

tecnologia_principio
ESILATO DE NINTEDANIBE                                                                                                      419
ARIPIPRAZOL                                                                                                                 388
TETRAIDROCANABINOL + CANABIDIOL                                                                                             387
PEMBROLIZUMABE                                                                                                              383
DUPILUMABE                                                                                                                  369
                                                                                                                           ... 
CLORIDRATO DE LISINA + CLORIDRATO DE PIRIDOXINA + CLORIDRATO DE TIAMINA + PANTOTENATO DE CÁLCIO + ÁCIDO GAMAMINOBUTÍRICO      1
FLUOROURACIL                                                                       